In [1]:
!pip install onnx
!pip install onnxscript

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 640.9/640.9 kB 8.6 MB/s eta 0:00:00


# Imports

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from PIL import Image

In [6]:
#Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
import os

os.environ['TORCH_LOGS'] = '+dynamo'
os.environ['TORCHDYNAMO_VERBOSE'] = '1'

# My Model

In [3]:
class StudentModel(nn.Module):
    def __init__(self, scale_factor=4):
        super(StudentModel, self).__init__()
        self.upsample = nn.Sequential(
            nn.Conv2d(3, 3 * (scale_factor ** 2), kernel_size=3, stride=1, padding=1),
            nn.PixelShuffle(scale_factor)
        )
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1)

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.upsample(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.relu(x)
        return x

In [9]:
torch_model = StudentModel()
#Get image size
sample_image_path = '/content/drive/MyDrive/SuperResolution/SuperResolution/ESRGAN/train/0024000/0023975x4.png'
sample_image = Image.open(sample_image_path)
input_size = sample_image.size  # This will give you (width, height)

torch_input = torch.randn(1, 3, input_size[1], input_size[0])
onnx_program = torch.onnx.dynamo_export(torch_model, torch_input)

/usr/local/lib/python3.10/dist-packages/torch/onnx/_internal/exporter.py:136: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(


In [12]:
onnx_program.save("/content/drive/MyDrive/Projects/ULX3s-Superresolution-CNN/SuperResolution_FEATUREMAPS_V2.onnx")